In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import random
import time
import pandas as pd
from getpass import getpass
import json

In [2]:
import pandas as pd
import os

In [19]:


# 文件路径
file_path = "/Users/zhouxiaoyu/Desktop/golem/id_v2/Italiano_with_urls.csv"

# 加载 CSV 文件并命名为 source_data
try:
    source_data = pd.read_csv(file_path)
    
    # 打印 DataFrame 的基本信息
    print("source_data Loaded Successfully!")
    print(f"\nColumns in source_data:\n{source_data.columns}")
    print("\nFirst few rows of source_data:")
    print(source_data.head())
    
    # 打印数据类型和摘要
    print("\nsource_data Info:")
    print(source_data.info())
except Exception as e:
    print(f"Error loading file: {e}")

source_data Loaded Successfully!

Columns in source_data:
Index(['story_id', 'tag_list', 'category', 'fandoms', 'word_count', 'url'], dtype='object')

First few rows of source_data:
   story_id                                           tag_list  \
0   4979362  ['actionadventure', 'love', 'magic', 'time tra...   
1  11958180  ['alternate universe  modern setting', 'fallin...   
2  20208829  ['alternate universe  modern setting', 'fallin...   
3  20952794  ['alternate universe  modern setting', 'first ...   
4  22653217  ['alternate universe  modern setting', 'cuddli...   

                                  category  \
0  ['character', 'event', 'world setting']   
1  ['character', 'event', 'world setting']   
2  ['character', 'event', 'world setting']   
3  ['character', 'event', 'world setting']   
4  ['character', 'event', 'world setting']   

                                             fandoms  word_count  \
0  Doctor Who, Doctor Who & Related Fandoms, Doct...       12574   
1       

In [20]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def login(driver, username, password):
    driver.get("https://archiveofourown.org/") 
    
    try:
        # 点击登录下拉菜单
        login_dropdown = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'login-dropdown'))
        )
        login_dropdown.click()
        
        # 输入用户名
        username_field = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'user_session_login_small'))
        )
        username_field.send_keys(username)
        
        # 输入密码
        password_field = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'user_session_password_small'))
        )
        password_field.send_keys(password)
        
        # 等待登录按钮可见
        login_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "commit"))
        )
        
        # 使用 JavaScript 执行点击操作
        driver.execute_script("arguments[0].click();", login_button)
        
        print("Logged in successfully")
    except Exception as e:
        print("Error during login:", e)
        driver.quit()

username = "Xiaoyu_GolemLab"
password = "ZXYmm123"

In [21]:
# Define sleep time to prevent banning from server
def sleep():
    sleep_time = 0 + random.randint(0, 10) + random.random()
    time.sleep(sleep_time)

# List of URLs to scrape
urls = source_data.url.to_list()

# Set up Chrome options
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")

# Sample user agents to rotate between requests
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 13_5 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0"
]

def setup_driver(user_agent):
    """Set up WebDriver with a specified user agent."""
    options = webdriver.ChromeOptions()
    options.add_argument(f"user-agent={user_agent}")
    return webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

In [22]:
def extract_data_from_page(url):
    try:
         driver.get(url)

        # retrieve the page source for parsing
         page_source = driver.page_source
    
        # Use BeautifulSoup to parse the page
         soup = BeautifulSoup(page_source, 'html.parser')

        # Locate the main section with meta information
         meta_section = soup.find("dl", class_="work meta group")
         if not meta_section:
            print(f"Meta section not found on page: {url}")
            return None

        # Extract Language
         language = meta_section.find("dd", class_="language")
         language_text = language.get_text(strip=True) if language else "N/A"

        # Extract Stats (Published, Updated, Words, etc.)
         stats_section = meta_section.find("dd", class_="stats")
         stats_data = {}
         if stats_section:
            # Iterate over each stat type
            for stat in stats_section.find_all("dt"):
                stat_type = stat.get_text(strip=True).rstrip(":")
                stat_value = stat.find_next_sibling("dd").get_text(strip=True)
                stats_data[stat_type] = stat_value
         else:
             print(f"Stats section not found on page: {url}")
        
        # Organize and return the extracted data
         data = {
            "URL": url,
            "Language": language_text,
            "Stats": stats_data,
        }
        
    except Exception as e:
        print(f"Error: {e}")
        print(f"Skipping URL: {url}")

    sleep() 
    
    return data

In [23]:
# Choose a random user agent for each URL
user_agent = random.choice(user_agents)
print(f"Using User Agent: {user_agent}")

# Initialize the driver with the chosen user agent
driver = setup_driver(user_agent)

# Login
login(driver, username, password)

# Iterate over URLs and scrape data
all_data = []
for url in urls:
    page_data = extract_data_from_page(url)
    if page_data:
        all_data.append(page_data)

Using User Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36
Logged in successfully


In [24]:
for entry in all_data:
    print(entry)

{'URL': 'https://archiveofourown.org/works/4979362?view_full_work=true', 'Language': 'Italiano', 'Stats': {'Published': '2015-10-11', 'Words': '12,574', 'Chapters': '1/1', 'Kudos': '5', 'Hits': '204'}}
{'URL': 'https://archiveofourown.org/works/11958180?view_full_work=true', 'Language': 'Italiano', 'Stats': {'Published': '2017-08-31', 'Updated': '2017-12-30', 'Words': '40,635', 'Chapters': '17/?', 'Kudos': '6', 'Hits': '416'}}
{'URL': 'https://archiveofourown.org/works/20208829?view_full_work=true', 'Language': 'Italiano', 'Stats': {'Published': '2019-08-11', 'Words': '1,711', 'Chapters': '1/1', 'Comments': '2', 'Kudos': '11', 'Bookmarks': '3', 'Hits': '77'}}
{'URL': 'https://archiveofourown.org/works/20952794?view_full_work=true', 'Language': 'Italiano', 'Stats': {'Published': '2019-10-08', 'Words': '1,595', 'Chapters': '1/1', 'Kudos': '11', 'Bookmarks': '3', 'Hits': '79'}}
{'URL': 'https://archiveofourown.org/works/22653217?view_full_work=true', 'Language': 'Italiano', 'Stats': {'Pub

In [25]:
import pandas as pd
import numpy as np

# 为 source_data 添加空列
source_data['kudos'] = np.nan
source_data['hits'] = np.nan

# 更新 kudos 和 hits 数据到 source_data
for entry in all_data:
    url = entry['URL']
    kudos = entry['Stats'].get('Kudos', 'N/A')  # 获取 Kudos，若不存在则填 'N/A'
    hits = entry['Stats'].get('Hits', 'N/A')    # 获取 Hits，若不存在则填 'N/A'
    
    # 找到对应 URL 的行并更新
    source_data.loc[source_data['url'] == url, 'kudos'] = kudos
    source_data.loc[source_data['url'] == url, 'hits'] = hits

# 查看更新后的数据
print(source_data)

     story_id                                           tag_list  \
0     4979362  ['actionadventure', 'love', 'magic', 'time tra...   
1    11958180  ['alternate universe  modern setting', 'fallin...   
2    20208829  ['alternate universe  modern setting', 'fallin...   
3    20952794  ['alternate universe  modern setting', 'first ...   
4    22653217  ['alternate universe  modern setting', 'cuddli...   
..        ...                                                ...   
116  36643324  ['alternate universe  canon divergence', 'not ...   
117  39554313                               ['love confessions']   
118  39747534                                ['character study']   
119  41564259                                        ['kissing']   
120  41614671                                    ['hurtcomfort']   

                                    category  \
0    ['character', 'event', 'world setting']   
1    ['character', 'event', 'world setting']   
2    ['character', 'event', 'world sett

In [26]:
# 保存为 CSV 文件
csv_filename = 'source_data.csv'
source_data.to_csv(csv_filename, index=False)

print(f"DataFrame has been saved to '{csv_filename}'")

DataFrame has been saved to 'source_data.csv'
